In [1]:
# from transformers import BertConfig, BertForMaskedLM, BertTokenizer
# from transformers import PreTrainedTokenizer
# from transformers import get_linear_schedule_with_warmup
# import torch
# from torch.utils.data import Dataset, DataLoader
# from torch.utils.data import random_split
# from torch.nn.functional import pad
# from datasets import load_dataset
from tqdm.auto import tqdm
# from itertools import cycle
# import wandb


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [2]:
import copy
from collections import defaultdict

# import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# from   matplotlib import colors
# import seaborn as sns

import json
import os
from pathlib import Path
from glob import glob
import gc
import pickle
import random
import sys

# from subprocess import Popen, PIPE, STDOUT

# re-arc data

In [3]:
sys.path.append('/home/coder/project/re-arc')
from main import get_generators


all_generators_map = get_generators()

In [4]:
class taskdict(dict):
    def __hash__(self):
        return hash(self['input'])


def generate(n_train_samples_per_task=7500, n_test_samples_per_task=2500, ub=1.0):
    n_tasks = len(all_generators_map)
    n_total_samples_per_task = n_train_samples_per_task + n_test_samples_per_task

    # max_board_numel = 100

    # def is_exceed_sample(sample):
    #     return (
    #         np.size(sample['input']) > max_board_numel
    #         or
    #         np.size(sample['output']) > max_board_numel
    #     )

    # def is_exceed_task(task):
    #     is_exceed_train = any(map(is_exceed_sample, task['train']))
    #     is_exceed_test = any(map(is_exceed_sample, task['test']))
    #     return is_exceed_train or is_exceed_test

    def convert_board_tolist(board):
        board = np.asarray(board).tolist()
        return board

    def convert_task_sample_tolist(task_sample):
        task_sample['input'] = convert_board_tolist(task_sample['input'])
        task_sample['output'] = convert_board_tolist(task_sample['output'])
        return task_sample

    def convert_board_totuple(board):
        board = convert_board_tolist(board)
        board = tuple([tuple(row) for row in board])
        return board

    def convert_task_sample_totuple(task_sample):
        task_sample['input'] = convert_board_totuple(task_sample['input'])
        task_sample['output'] = convert_board_totuple(task_sample['output'])
        return task_sample

    # def convert_datasample_tolist(datasample):
    #     for i, train_task_sample in enumerate(datasample['train']):
    #         train_task_sample = convert_task_sample_tolist(train_task_sample)
    #         datasample['train'][i] = train_task_sample

    #     for i, test_task_sample in enumerate(datasample['test']):
    #         test_task_sample = convert_task_sample_tolist(test_task_sample)
    #         datasample['test'][i] = test_task_sample

    #     return datasample

    def is_equal(sample1, sample2):
        input1 = sample1['input']
        input2 = sample2['input']

        nrow1, ncol1 = np.shape(input1)
        nrow2, ncol2 = np.shape(input2)

        if ( (nrow1 != nrow2) or (ncol1 != ncol2) ):
            return False

        if any((row1 != row2) for (row1, row2) in zip(input1, input2)):
            return False

        return True


    data = dict()

    all_keys = list(all_generators_map.keys())

    for i, task_key in enumerate(tqdm(all_keys, desc='tasks')):
        task_generator = all_generators_map[task_key]

        samples = []

        while len(samples) < n_total_samples_per_task:
            try:
                one_sample = task_generator(0, ub)
            except IndexError:
                continue
            except TypeError:  # (266, aabf363d)
                continue
            else:
                samples.append(one_sample)

        # train_samples = samples[:n_train_samples_per_task]
        # test_samples = samples[:n_train_samples_per_task: n_train_samples_per_task + n_test_samples_per_task]

        # collect train tasks for further filtration
        train_range = slice(0, n_train_samples_per_task)
        train_samples = defaultdict(list)
        for one_sample in samples[train_range]:
            hash_tag = np.shape(one_sample['input'])
            hash_sample = taskdict(convert_task_sample_totuple(one_sample))
            train_samples[tuple(hash_tag)].append(hash_sample)

        # filter train data off repetitive samples
        train_samples = {tag: set(values) for tag, values in train_samples.items()}

        # collect test tasks for further filtration
        test_range = slice(n_train_samples_per_task, n_train_samples_per_task + n_test_samples_per_task)
        test_samples = defaultdict(list)
        for one_sample in samples[test_range]:
            hash_tag = np.shape(one_sample['input'])
            hash_sample = taskdict(convert_task_sample_totuple(one_sample))
            test_samples[tuple(hash_tag)].append(hash_sample)

        # filter test data off repetitive samples
        test_samples = {tag: set(values) for tag, values in test_samples.items()}

        # exclude train samples that present in test data
        for tag in test_samples.keys():
            test_samples[tag] -= train_samples.get(tag, set())

        # format data back to lists of samples
        train_samples = [one_sample for sample_group in train_samples.values() for one_sample in sample_group]
        test_samples = [one_sample for sample_group in test_samples.values() for one_sample in sample_group]

        # shuffle data
        random.shuffle(train_samples)
        random.shuffle(test_samples)

        # unique_ids = []
        # for i, one_sample in enumerate(test_samples):
        #     if any(is_equal(one_sample, other_sample) for other_sample in train_samples):
        #         continue
        #     unique_ids.append(i)

        # if len(unique_ids) < 1000:
        #     print(len(unique_ids))

        # test_samples = [test_samples[i] for i in unique_ids]

        # train_samples = [convert_task_sample_tolist(one_sample) for one_sample in train_samples]
        # test_samples = [convert_task_sample_tolist(one_sample) for one_sample in test_samples]

        # format back to ordinary dict
        train_samples = [dict(one_sample) for one_sample in train_samples]
        test_samples = [dict(one_sample) for one_sample in test_samples]

        print(f'id: {i:03d}, key: {task_key}. #train: {len(train_samples)}, #test: {len(test_samples)}')

        data[task_key] = {
            'train': train_samples,
            'test': test_samples,
        }

    return data

In [5]:
data = generate(n_train_samples_per_task=3000, n_test_samples_per_task=1000, ub=1.0)

tasks:   0%|          | 0/400 [00:00<?, ?it/s]

id: 000, key: 007bbfb7. #train: 2758, #test: 848
id: 001, key: 00d62c1b. #train: 3000, #test: 1000
id: 002, key: 017c7c7b. #train: 2972, #test: 976
id: 003, key: 025d127b. #train: 2999, #test: 999
id: 004, key: 045e512c. #train: 3000, #test: 1000
id: 005, key: 0520fde7. #train: 3000, #test: 1000
id: 006, key: 05269061. #train: 2990, #test: 995
id: 007, key: 05f2a901. #train: 3000, #test: 1000
id: 008, key: 06df4c85. #train: 3000, #test: 1000
id: 009, key: 08ed6ac7. #train: 3000, #test: 1000
id: 010, key: 09629e4f. #train: 3000, #test: 1000
id: 011, key: 0962bcdd. #train: 3000, #test: 1000
id: 012, key: 0a938d79. #train: 3000, #test: 1000
id: 013, key: 0b148d64. #train: 3000, #test: 1000
id: 014, key: 0ca9ddb6. #train: 3000, #test: 1000
id: 015, key: 0d3d703e. #train: 2997, #test: 998
id: 016, key: 0dfd9992. #train: 3000, #test: 1000
id: 017, key: 0e206a2e. #train: 3000, #test: 1000
id: 018, key: 10fcaaa3. #train: 3000, #test: 1000
id: 019, key: 11852cab. #train: 3000, #test: 1000
id: 0

In [6]:
# with open('data/rearc1M.json', 'w') as f:
#     json.dump(data, f, indent=2)

In [8]:
with open('../data/rearc_400x4k_gross.pkl', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle

with open('../data/rearc_400x4k_gross.pkl', 'rb') as f:
    other_data = pickle.load(f)

In [6]:
len(other_data)

400

In [7]:
N_SAMPLES_PER_TASK = 5_000

sampled_data = dict()

for task_key, task_data in tqdm(other_data.items()):
    task_samples = []

    train_samples = task_data['train']
    test_samples = task_data['test']

    used_ids = set()
    while len(task_samples) < N_SAMPLES_PER_TASK:
        train_ids = np.random.choice(len(train_samples), size=4, replace=False)
        test_id = np.random.choice(len(test_samples))

        all_ids = tuple(train_ids.tolist() + [test_id])
        if all_ids in used_ids:
            continue
        used_ids.add(all_ids)

        task_samples.append({
            'train': [train_samples[idx] for idx in train_ids],
            'test': [test_samples[test_id]],
        })

    sampled_data[task_key] = task_samples

  0%|          | 0/400 [00:00<?, ?it/s]

In [8]:
with open('../data/rearc_400x4k_gross_sampled1.pkl', 'wb') as f:
    pickle.dump(sampled_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
import pickle

with open('../data/rearc_400x4k_gross_sampled1.pkl', 'rb') as f:
    other_sampled_data = pickle.load(f)